In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import ast
import re
import joblib
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, hamming_loss, jaccard_score

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Предобработка датасета

In [ ]:
data_all = pd.read_csv('/content/drive/MyDrive/Годовой проект /habr.csv')

In [ ]:
data = data_all.sample(30000, random_state=42).copy() # Посмотрим результаты модели на части данных (нехватка памяти)

In [ ]:
data.head()

,id,text,keywords,hubs,username,reading_time,title,time,status
2964,396629,В отчете впервые появились HDD объемом 8 ТБКом...,"['жесткие диски', 'Backblaze', 'статистика']","['Статистика в IT', 'Компьютерное железо', 'На...",marks,3.0,Статистика Backblaze: определяем лучший HDD во...,2016-08-02 18:31:08,ok
57555,525612,Grafana Labs с гордостью представляет простую ...,"['grafana', 'tracing', 'prometheus']","['Блог компании Слёрм', 'DevOps', 'IT-инфрастр...",Finnix,3.0,"Представлена Grafana Tempo, широко масштабируе...",2020-10-29 12:11:36,ok
75874,568966,В Java 8 появилось два вида функциональных выр...,"['java', 'функциональный интерфейс', 'invokedy...","['Java', 'Программирование']",tagir_valeev,5.0,Как сделать ссылки на методы дружелюбными для ...,2021-07-21 17:36:21,ok
81801,582732,Вице-президент по мировым вопросам Facebook Ни...,"['instagram', 'facebook', 'соцсети', 'модериро...","['Социальные сети', 'IT-компании']",ondlusha,1.0,Instagram будет предлагать подросткам «отвлечь...,2021-10-11 09:51:43,ok
85560,591353,30 ноября в 16:00 мы проведемвебинар про NoSQL...,"['вебинар', 'redis', 'базы данных', 'субд']","['Блог компании Selectel', 'Базы данных', 'Обл...",andreyzaytsev,1.0,Запись вебинара по базам данных. Redis: как ра...,2021-11-25 11:50:50,ok


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 2964 to 37587
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            30000 non-null  int64  
 1   text          29932 non-null  object 
 2   keywords      30000 non-null  object 
 3   hubs          30000 non-null  object 
 4   username      29917 non-null  object 
 5   reading_time  29999 non-null  float64
 6   title         30000 non-null  object 
 7   time          30000 non-null  object 
 8   status        30000 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.3+ MB


In [ ]:
data.isna().sum()

,0
id,0
text,68
keywords,0
hubs,0
username,83
reading_time,1
title,0
time,0
status,0


In [ ]:
data['text'].fillna('', inplace=True)
data['username'] = data['username'].astype("string").fillna('')
data['reading_time'].fillna(data['reading_time'].median(), inplace=True)

data['hubs'] = data['hubs'].apply(ast.literal_eval)
data['keywords'] = data['keywords'].apply(ast.literal_eval)

data['combined_text'] = data['text']+' '+data['title']+' '+data['keywords'].apply(lambda x: ' '.join(x))

/tmp/ipython-input-996675763.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['text'].fillna('', inplace=True)
/tmp/ipython-input-996675763.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

In [ ]:
def clean_text_basic(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()
  text = re.sub(r'<[^>]+>', ' ', text)
  text = re.sub(r'http\S+|www\S+|https\S+', ' ', text, flags=re.MULTILINE)
  text = re.sub(r'\S*@\S*\s?', ' ', text)
  text = re.sub(r'[^\w\s.,!?;:()-]', ' ', text)
  text = ' '.join(text.split())
  return text

data['clean_text'] = data['combined_text'].apply(clean_text_basic)
data.drop(columns=['combined_text'], inplace=True)

Небольшой feature engineering

In [ ]:
data['text_length'] = data['text'].str.len()
data['word_count'] = data['text'].str.split().str.len()

In [ ]:
author_stats = data['username'].value_counts()
data['author_popularity'] = data['username'].map(author_stats)

Попытаемся отфильтровать хабы


In [ ]:
all_hubs = [hub for sublist in data['hubs'] for hub in sublist]
hub_counts = pd.Series(all_hubs).value_counts()

MIN_HUB_FREQ = 10
common_hubs = hub_counts[hub_counts >= MIN_HUB_FREQ].index.tolist()

def filter_hubs(hubs_list):
    return [hub for hub in hubs_list if hub in common_hubs]

data['filtered_hubs'] = data['hubs'].apply(filter_hubs)
data = data[data['filtered_hubs'].apply(len) > 0].copy()

In [ ]:
russian_stopwords = stopwords.words('russian')

In [ ]:
vectorizer = TfidfVectorizer(
  max_features=3000,
    min_df=3,
    max_df=0.85,
    ngram_range=(1, 2),
    stop_words=russian_stopwords,
    analyzer='word'
)

X = vectorizer.fit_transform(data['clean_text'])

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['filtered_hubs'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
model = OneVsRestClassifier(
    LinearSVC(
        C=0.7,
        class_weight='balanced',
        max_iter=10000,
        random_state=42,
        dual=False
    ),
    n_jobs=-1
)


model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(C=0.7, class_weight='balanced',
                                        dual=False, max_iter=10000,
                                        random_state=42),
                    n_jobs=-1)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
micro_f1 = f1_score(y_test, y_pred, average='micro')
macro_f1 = f1_score(y_test, y_pred, average='macro')

micro_f1 = f1_score(y_test, y_pred, average='micro')
macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

print(f"Micro F1: {micro_f1:.4f}")
print(f"Macro F1: {macro_f1:.4f}")

micro_precision = precision_score(y_test, y_pred, average='micro', zero_division=0)
micro_recall = recall_score(y_test, y_pred, average='micro', zero_division=0)
macro_precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
macro_recall = recall_score(y_test, y_pred, average='macro', zero_division=0)

print(f"Micro Precision: {micro_precision:.4f}")
print(f"Micro Recall: {micro_recall:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")

hamming = hamming_loss(y_test, y_pred)
jaccard_micro = jaccard_score(y_test, y_pred, average='samples')
jaccard_macro = jaccard_score(y_test, y_pred, average='macro', zero_division=0)

print(f"Hamming Loss: {hamming:.4f}")
print(f"Jaccard Score (samples): {jaccard_micro:.4f}")
print(f"Jaccard Score (macro): {jaccard_macro:.4f}")


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Micro F1: 0.4404
Macro F1: 0.2888
Micro Precision: 0.3609
Micro Recall: 0.5648
Macro Precision: 0.3230
Macro Recall: 0.3130
Hamming Loss: 0.0079
Jaccard Score (samples): 0.3058
Jaccard Score (macro): 0.1988


Пример предсказания статьи

In [ ]:
def predict_hubs(text, top_n=3):
    text_vec = vectorizer.transform([text])
    predictions = model.decision_function(text_vec)[0]

    top_indices = np.argsort(predictions)[-top_n:][::-1]
    top_hubs = [(mlb.classes_[i], predictions[i]) for i in top_indices]

    return top_hubs

sample_text = data.iloc[0]['clean_text']
predicted = predict_hubs(sample_text)
print("Пример предсказания:")
for hub, score in predicted:
    print(f"{hub}: {score:.3f}")

Пример предсказания:
Хранение данных: 1.291
Накопители: 0.998
Статистика в IT: 0.952


In [ ]:
model_data = {
    'vectorizer': vectorizer,
    'mlb': mlb,
    'model': model,
    'common_hubs': common_hubs
}

joblib.dump(model_data, 'model.pkl')
print("Модель сохранена")

Модель сохранена
